In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import time

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 이 파일이 api 호출해야하는 총 데이터 (원본)
name = pd.read_csv('/content/drive/Shareddrives/2021 팜 경진대회/2 데이터/0 가공완료 데이터/서울시 버스정류장 중복 제거 _통합.csv', encoding='cp949')
name = name[['정류소명', '행정동']]
name

,정류소명,행정동
0,청암자이아파트,원효로제2동
1,청암동강변삼성아파트,원효로제2동
2,청심경로당,원효로제2동
3,원효2동주민센터,원효로제2동
4,산천동,원효로제2동
...,...,...
11028,면목지구대,면목본동
11029,새마을금고,면목제3.8동
11030,공판장,면목제3.8동
11031,진주빌라.바다약국,면목제3.8동


인덱싱 부분 ( 그냥 저 숫자 그대로 밑에 imsi 인덱싱에 넣기 )
- 지윤 : 1050 ~ 2050 / 2050 ~ 3050
- 지수 : 3050 ~ 4050 / 4050 ~ 5050
- 다빈 : 5050 ~ 6050 / 6050 ~ 7050
- 민서 : 0~1050 / 7050 ~ 8050 / 8050 ~ 9050 / 9050 ~ 10050 / 10050 ~ 11033

---

< key 불러오는 방법 >
1. 공공데이터 포털에 아이디 비번 로그인
2. 마이페이지 들어가기
3. [승인]서울특별시_정류소정보조회 서비스 누르기
4. 일반 인증키(Encoding) 적힌 부분에 있는 키를 Key = 값에 넣기

In [ ]:
# 이 imsi가 각자 1000개씩 돌려야해서 인덱싱 해야하는 부분 -> 각자 번호에 맞게 고치기
imsi = name.iloc[50:1050, 0]
imsi

In [ ]:
# 여기서 세번째 줄에 key 부분 바꿔서 돌리기
apicall=[]
for i in imsi:
  key = 'Ig305vjzG01urQJR7VkY6XXiBDTkRkJ5b8Jar3cyq5Q6tl4neoFjq5u%2FhitQ0CB1CB%2BfD3PmaJa2Xo6ZWVaqXw%3D%3D' 
  stSrch=i

  queryParams = 'ServiceKey='+key+'&stSrch='+stSrch
  url = 'http://ws.bus.go.kr/api/rest/stationinfo/getStationByName?'+queryParams

  req = requests.get(url)
  html = req.text 

  soup = BeautifulSoup(html, 'html.parser')
  arsid = soup.select('arsid')
  stid = soup.select('stid')
  stnm= soup.select('stnm')
  tmx= soup.select('tmx')
  tmy= soup.select('tmy')

  all = list(zip(arsid,stid,stnm,tmx,tmy))
  for arsid,stid,stnm,tmx,tmy in all:
      apicall.append([arsid.text, stid.text, stnm.text, tmx.text, tmy.text])
      
apicall

In [ ]:
# 밑에 저장할 이름에 apicall_(시작인덱스)(종료인덱스) 
# ex) 지윤이 첫번째 이름 apicall_10502050
apicall = pd.DataFrame(data=apicall, columns=['arsid', 'stid', 'stnm', 'x', 'y'])
apicall.to_csv('/content/drive/Shareddrives/2021 팜 경진대회/3 코드/3 버스노선 관련/apicall_OOOO.csv', encoding='cp949')